In [12]:
import sagemaker
import boto3
from sagemaker.huggingface.model import HuggingFacePredictor
import json

In [13]:
# AWS login to use sagemaker endpoints
session = boto3.Session(profile_name='vrt-analytics-engineer-nonsensitive')
sagemaker_session = sagemaker.Session(boto_session=session)
role = sagemaker.get_execution_role(sagemaker_session=sagemaker_session)

# Vector store endpoint
endpoint_name = "faiss-endpoint-1722716142"
faiss_vector_store = sagemaker.predictor.Predictor(endpoint_name,sagemaker_session=sagemaker_session)
assert faiss_vector_store.endpoint_context().properties['Status'] == 'InService'

# LLM endpoint
llm = HuggingFacePredictor(endpoint_name="huggingface-pytorch-tgi-inference-2024-07-30-20-23-30-977",sagemaker_session=sagemaker_session)

In [14]:
question = "iets over vrijheid"

In [15]:
# Fetch content
payload = json.dumps({
    "text": question,
})

out = faiss_vector_store.predict(
    payload,
    initial_args={"ContentType": "application/json", "Accept": "application/json"}
)
out = json.loads(out)

In [16]:
out

{'0': {'page_content': "Wat is vrijheid van denken? Met antwoorden van Helder De Schutter, Rob Riemen en Wies De Graeve. Vrijheid van denken is \nde mogelijkheid hebben om niet gehinderd \ndoor wat anderen denken of vinden, je eigen gedachtes te vormen. Dus volledig zelf kunnen kiezen \nwaaraan je aan het denken bent. Ook als het verschrikkelijk rare \ndingen zijn die je zou bedenken. Het is een heel belangrijke component  \nvan de liberale democratie. Het is een vrijheidsbeginsel. En het is iets \nwaarvan bijna iedereen vindt dat je er nooit aan mag raken. Je vrijheid is compleet, totaal. Wat voor belachelijke dingen je denkt, je bent honderd percent vrij \nom dat te doen. Er is een bekend beginsel \nin het liberale denken. John Stuart Mill, \nhet schadebeginsel: mijn vrijheid eindigt \nwaar ik jou schaad. Maar de vrijheid van denken \nwordt niet beperkt door dat schadebeginsel. Het staat je open \nom te denken wat je wilt, want je schaadt me niet door te denken \nwat jij aan het denk

In [17]:
# Generate prompt
system_prompt = """
Je bent een hulpvaardige assistent die Nederlands spreekt.
Je krijgt vragen over de catalogus van programma's van het videoplatform VRT MAX. 
Je probeert mensen te helpen om programma's aan te bevelen die aansluiten bij hun vraag.
Daarvoor krijg je een beschrijving van een aantal programma's.
Aan jou om wat relevant is aan te bevelen.
Begin niet met het geven van jouw mening over de programma's. Begin direct met het aanbevelen van relevante content aan de gebruiker.
"""

user_prompt = ""
for item in out.keys():
    
    user_prompt += "Het programma met de naam " + out[item]["metadata"]["mediacontent_pagetitle_program"] +\
          " heeft volgende beschrijving: " + out[item]["metadata"]["mediacontent_page_description"] 
    
    # + " en in het programma wordt er onder meer het volgende gezegd dat misschien relevant is voor de vraag: " + \out[item]["page_content"]

user_prompt += " Tot zover alle informatie over de programmas."

user_prompt += " Gelieve met die informatie een antwoord te geven op volgende vraag: " + question

In [18]:
# Ask question
# Prompt to generate
messages=[
    { "role": "system", "content": system_prompt },
    { "role": "user", "content": user_prompt }
  ]

# Generation arguments
parameters = {
    "model": "meta-llama/Meta-Llama-3-8B-Instruct", # placeholder, needed
    "top_p": 0.6,
    "temperature": 0.9,
    "max_tokens": 512,
    "stop": ["<|eot_id|>"],
}

chat = llm.predict({"messages" :messages, **parameters})

print(chat["choices"][0]["message"]["content"].strip())

Aan de hand van de beschrijvingen van de programma's Grote vragen en Winteruur lijkt het alsof deze programma's interessante en relevante inhoud hebben over het thema vrijheid. 

Ik zou de programma's Grote vragen en Winteruur aanbevelen om meer te weten te komen over het thema vrijheid. In Grote vragen worden verschillende kanten van het thema vrijheid besproken, zoals de vraag wanneer je werkelijk vrij bent en wat vrijheid van denken inhoudt. In Winteruur worden teksten van bekende auteurs besproken die over vrijheid schrijven, zoals Stefan Hertmans en Victor E. Frankl. 

Beide programma's lijken mij relevante en interessante content te bieden voor iemand die iets wil weten over het thema vrijheid.
